In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import MinMaxScaler


c:\venv_python_3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import xgboost as xgb
import lightgbm as lgbm
import catboost as cbt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Iris Dataset

In [83]:
# import some data to play with
# iris = datasets.load_iris()
breast_cancer = datasets.load_breast_cancer()

# Take the first two features. We could avoid this by using a two-dim dataset
# X = breast_cancer.data[:, :4]
X = breast_cancer.data
y = breast_cancer.target

In [84]:
print("X shape")
print(X.shape)
print("Y shape")
print(y.shape)


X shape
(569, 30)
Y shape
(569,)


In [85]:
# x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4096)
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4096, train_size=0.8)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=4096, train_size=0.8)

In [86]:
scaler = MinMaxScaler()

scaler = scaler.fit(x_train)


In [87]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

In [88]:
# data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
# target = raw_df.values[1::2, 2]
# x_train, x_test, y_train, y_test = train_test_split(data, target, random_state=4096)

# SVM - Linear

In [89]:

svm_classifier = SVC(kernel='linear')
svm_classifier = svm_classifier.fit(x_train, y_train)

In [90]:
y_pred = svm_classifier.predict(x_test)



print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))


Confusion Matrix
 [[40  2]
 [ 0 72]]
Accuracy  0.9824561403508771


# SVM - Kernel

In [91]:
param_grid = [
    {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
    {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
                        'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
]

In [92]:

svm_classifier = SVC(kernel='rbf')
# svm_classifier = svm_classifier.fit(x_train, y_train)

grid_search = GridSearchCV(svm_classifier, param_grid, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_svm_classifier = grid_search.fit(x_train, y_train)



Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=10.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END ..............................C=30.0, kernel=linear; total time=   0.0s
[CV] END .............................C=100.0, kernel=linear; total time=   0.0s
[CV] END .............................C=100.0, 

In [93]:
grid_search.best_params_


{'C': 3.0, 'gamma': 0.3, 'kernel': 'rbf'}

In [94]:
# y_pred = svm_classifier.predict(x_test)
y_pred = best_svm_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[40  2]
 [ 0 72]]
Accuracy  0.9824561403508771


# TabNet

In [124]:
tabnet_classifier = TabNetClassifier()
# clf = TabNetMultiTaskClassifier()
tabnet_classifier.fit(
  X_train=x_train, y_train=y_train,
  eval_set = [(x_valid, y_valid)],
  patience=1000,
  max_epochs=30,
  batch_size=16,
)

c:\venv_python_3.10\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.57925 | val_0_auc: 0.89525 |  0:00:00s
epoch 1  | loss: 0.33204 | val_0_auc: 0.96698 |  0:00:01s
epoch 2  | loss: 0.28655 | val_0_auc: 0.9644  |  0:00:02s
epoch 3  | loss: 0.26506 | val_0_auc: 0.98968 |  0:00:03s
epoch 4  | loss: 0.23584 | val_0_auc: 0.9711  |  0:00:04s
epoch 5  | loss: 0.30137 | val_0_auc: 0.96233 |  0:00:05s
epoch 6  | loss: 0.30701 | val_0_auc: 0.96698 |  0:00:06s
epoch 7  | loss: 0.25692 | val_0_auc: 0.9773  |  0:00:06s
epoch 8  | loss: 0.17528 | val_0_auc: 0.98555 |  0:00:07s
epoch 9  | loss: 0.20487 | val_0_auc: 0.98246 |  0:00:08s
epoch 10 | loss: 0.26974 | val_0_auc: 0.99226 |  0:00:09s
epoch 11 | loss: 0.19468 | val_0_auc: 0.98658 |  0:00:10s
epoch 12 | loss: 0.21299 | val_0_auc: 0.98555 |  0:00:11s
epoch 13 | loss: 0.30767 | val_0_auc: 0.98607 |  0:00:12s
epoch 14 | loss: 0.2149  | val_0_auc: 0.98555 |  0:00:13s
epoch 15 | loss: 0.18563 | val_0_auc: 0.98865 |  0:00:14s
epoch 16 | loss: 0.19684 | val_0_auc: 0.98865 |  0:00:15s
epoch 17 | los

c:\venv_python_3.10\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [125]:
y_pred = tabnet_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[40  2]
 [ 1 71]]
Accuracy  0.9736842105263158


# XGBoost

In [97]:
num_round = 3
xgb_classifier = xgb.XGBClassifier()

xgb_parameters ={'max_depth' : [3,4,5,6] , 'n_estimators': [12,24,32], 'learning_rate':[0.01, 0.1], 'gamma': [0.5, 1, 2], 'random_state':[99]}

In [98]:
# xgb_classifier=xgb_classifier.fit(x_train, y_train)

xgb_grid_search = GridSearchCV(xgb_classifier, xgb_parameters, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_xgb_classifier = xgb_grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=12, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; total time=   0.0s
[CV] END gamma=0.5, learning_rate=0.01, max_depth=3, n_estimators=24, random_state=99; tot

In [99]:

xgb_grid_search.best_params_

{'gamma': 2,
 'learning_rate': 0.1,
 'max_depth': 4,
 'n_estimators': 12,
 'random_state': 99}

In [100]:
# y_pred = xgb_classifier.predict(x_test)
y_pred = best_xgb_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[39  3]
 [ 3 69]]
Accuracy  0.9473684210526315


# LightGBM

In [101]:
num_round = 3
lgbm_classifier = lgbm.LGBMClassifier()

In [102]:
lgbm_classifier=lgbm_classifier.fit(x_train, y_train)

In [103]:

y_pred = lgbm_classifier.predict(x_test)



print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[39  3]
 [ 2 70]]
Accuracy  0.956140350877193


# CatBoost

In [104]:
num_round = 3
cbt_classifier = cbt.CatBoostClassifier()

In [111]:
cbt_parameters = {'depth'         : [3,4,5,6],
                'learning_rate' : [0.01,0.02,0.03],
                'n_estimators': [12,24,32], 
}

In [112]:
# cbt_classifier=cbt_classifier.fit(x_train, y_train)

cbt_grid_search = GridSearchCV(cbt_classifier, cbt_parameters, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_cbt_classifier = cbt_grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6832179	total: 1.24ms	remaining: 13.7ms
1:	learn: 0.6674665	total: 2.36ms	remaining: 11.8ms
2:	learn: 0.6511558	total: 3.49ms	remaining: 10.5ms
3:	learn: 0.6336802	total: 4.7ms	remaining: 9.4ms
4:	learn: 0.6246690	total: 5.7ms	remaining: 7.99ms
5:	learn: 0.6112076	total: 6.76ms	remaining: 6.76ms
6:	learn: 0.6031043	total: 7.74ms	remaining: 5.53ms
7:	learn: 0.5905653	total: 8.67ms	remaining: 4.33ms
8:	learn: 0.5764590	total: 9.92ms	remaining: 3.31ms
9:	learn: 0.5649093	total: 11ms	remaining: 2.19ms
10:	learn: 0.5515263	total: 11.9ms	remaining: 1.08ms
11:	learn: 0.5404431	total: 12.8ms	remaining: 0us
[CV] END .......depth=3, learning_rate=0.01, n_estimators=12; total time=   0.0s
0:	learn: 0.6815316	total: 1.13ms	remaining: 12.4ms
1:	learn: 0.6655712	total: 1.99ms	remaining: 9.94ms
2:	learn: 0.6493288	total: 2.84ms	remaining: 8.53ms
3:	learn: 0.6323032	total: 3.71ms	remaining: 7.42ms
4:	learn: 0.6201703	total: 4.54

In [113]:
cbt_grid_search.best_params_

{'depth': 5, 'learning_rate': 0.03, 'n_estimators': 32}

In [114]:
# y_pred = cbt_classifier.predict(x_test)
y_pred = best_cbt_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[39  3]
 [ 2 70]]
Accuracy  0.956140350877193


# Random Forest

In [115]:
rf_param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7],
    'criterion' :['gini', 'entropy']
}

In [116]:
num_round = 3
rf_classifier = RandomForestClassifier()

rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=5, scoring="neg_mean_squared_error", verbose=2)
best_rf_classifier = rf_grid_search.fit(x_train, y_train)



Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.1s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.1s
[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.1s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.1s
[CV] END criterion=gini, max_depth=4, max_features=auto, n_estimators=200; total time=   0.1s


...

c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=8, max_features=auto, n_estimators=500; total time=   0.4s


c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END criterion=entropy, max_depth=8, max_features=auto, n_estimators=500; total time=   0.5s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=200; total time=   0.1s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=500; total time=   0.4s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=500; total time=   0.4s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=500; total time=   0.4s
[CV] END criterion=entropy, max_depth=8, max_features=sqrt, n_estimators=500; total time=   0.4s
[CV] END criterion=entropy, ma

c:\venv_python_3.10\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [ ]:
# rf_classifier = rf_classifier.fit(x_train, y_train)

In [119]:
best_rf_classifier.best_params_

{'criterion': 'entropy',
 'max_depth': 6,
 'max_features': 'auto',
 'n_estimators': 500}

In [120]:

y_pred = best_rf_classifier.predict(x_test)


print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

print('Accuracy ', accuracy_score(y_test, y_pred))

Confusion Matrix
 [[40  2]
 [ 2 70]]
Accuracy  0.9649122807017544
